# Teorema di esistenza e unicità della soluzione

Dato un circuito composto da $l$ lati e $n$ nodi, abbiamo visto che, unendo le equazioni di Tableau alle equazioni costitutive dei componenti, è possibile scrivere il seguente sistema di $2l+n-1$ equazioni in $2l+n-1$ incognite:

$$
\begin{equation}
\left\{
\begin{array}{rcl}
A\ \boldsymbol{i}(t) & = & \boldsymbol{0} \\
\boldsymbol{v}(t) - A^T\ \boldsymbol{u}(t) & = & \boldsymbol{0} \\
M(t)\ \boldsymbol{v}(t) + N(t)\ \boldsymbol{i}(t) & = & \boldsymbol{z}(t)
\end{array}
\right.
\end{equation}
$$

Raggruppando le incognite nel vettore $w = [\boldsymbol u^T, \boldsymbol v^T, \boldsymbol i^T]^T$, possiamo scrivere il sistema in forma matriciale nel seguente modo:

$$
\begin{equation}
\underbrace{\left[
\begin{array}{c c c}
\mathbb{0}_{(n-1) \times (n-1)} & \mathbb{0}_{(n-1) \times l} & A\\
-A^T & \mathbb{1}_{l} & \mathbb{0}_{l \times l}\\
\mathbb{0}_{l \times (n-1)} & M(t) & N(t)
\end{array}
\right]}_{T(t)}
\underbrace{\left[
\begin{array}{c}
\boldsymbol u(t)\\
\boldsymbol v(t)\\
\boldsymbol i(t)
\end{array}
\right]}_{\boldsymbol w(t)}
=
\underbrace{\left[
\begin{array}{c}
\mathbb{0}_{(n-1) \times 1}\\
\mathbb{0}_{l \times 1}\\
{\bf z}(t)
\end{array}
\right]}_{\boldsymbol y(t)}
\end{equation}
$$

Il teorema di esistenza e unicità afferma quindi che un circuito ha una ed una sola soluzione se $\mathrm{det}(T(t)) \neq 0$. 

Consideriamo ora il seguente circuito e determiniamo il valore di tutte le grandezze elettriche, ossia potenziali di nodo, tensioni e correnti di lato.

<img src="figs/esistenza_unicita/ckt_1.png" width="400"/>

I vettori delle incognite sono dati da:

$$
\begin{eqnarray}
\boldsymbol u & = & \left[ u_1, u_2 \right]^T \\
\boldsymbol v & = & \left[ v_a, v_{R_1}, v_{R_2}, v_{cc,1} \right]^T \\
\boldsymbol i & = & \left[ i_a, i_{R_1}, i_{R_2}, i_{cc,1} \right]^T
\end{eqnarray}
$$

Disegnamo il grafico del circuito orientando i lati come le correnti contenute nel vettore $\boldsymbol i$:

<img src="figs/esistenza_unicita/graph.png" width="200"/>

La matrice di incidenza ridotta è data da

$$
\begin{equation}
A = 
\left[
\begin{array}{cccc}
0 & -1 & 1 & 1 \\
1 & 1 & 0 & 0
\end{array}
\right]
\end{equation}
$$

Le matrici $M$ e $N$ e il vettore $\boldsymbol z$ sono dati da

$$
\begin{equation}
M = \left[
\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}
$$

$$
\begin{equation}
N = \left[
\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & -R_1 & 0 & 0 \\
0 & 0 & -R_2 & 0 \\
0 & 0 & 0 & 0 \\
\end{array}
\right]
\end{equation}
$$

$$
\begin{equation}
\boldsymbol z = \left[
\begin{array}{c}
-I \\
0 \\
0 \\
0
\end{array}
\right]
\end{equation}
$$


Come prima cosa, definisco una funzione che, date le matrici $A$, $M$ e $N$ e il vettore $\boldsymbol z$, restituisce la matrice $T$ e il vettore $\boldsymbol y$.

In [1]:
import numpy as np

def make_T_y(A, M, N, z):
    n = A.shape[0] + 1
    l = A.shape[1]
    rows = 2 * l + n - 1
    cols = 2 * l + n - 1
    T = np.zeros((rows,cols))
    T[ :n-1 , -l: ] = A
    T[ n-1:n-1+l , :n-1 ] = -A.T
    T[ n-1:n-1+l , n-1:n-1+l ] = np.eye(l)
    T[ -l:, n-1:n-1+l ] = M
    T[ -l:, -l: ] = N
    y = np.zeros(rows)
    y[-l:] = z
    return T,y

Definisco i valori delle costanti e le matrici $A$, $M$, $N$ e il vettore $\boldsymbol z$.

In [2]:
R1 = 1   # [Ohm]
R2 = 2   # [Ohm]
I  = 1   # [A]

A = np.array([
         [0, -1, 1, 1],
         [1,  1, 0, 0]])

M = np.array([
         [0, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1]])

N = np.array([
         [1,   0,   0, 0],
         [0, -R1,   0, 0],
         [0,   0, -R2, 0],
         [0,   0,   0, 0]])

z = np.array([-I,0,0,0])

Costruisco la matrice $T$ e il vettore $\boldsymbol y$ e verifico che $\mathrm{det}(T(t)) \neq 0$.

In [3]:
T,y = make_T_y(A, M, N, z)
d = np.linalg.det(T)
print('det(T) = {}'.format(d))

det(T) = -2.0


Calcolo la soluzione del circuito risolvendo il sistema lineare:
    
$$
\boldsymbol w = T^{-1} \boldsymbol y
$$

In [4]:
w = np.linalg.solve(T,y)

Stampo i valori delle variabili:

In [5]:
var_names = ['u_1', 'u_2', 'v_a', 'v_R1', 'v_R2', 'v_cc', 'i_a', 'i_R1', 'i_R2', 'i_cc']
units = ['V', 'V', 'V', 'V', 'V', 'V', 'A', 'A', 'A', 'A']
for v,x,u in zip(var_names, w, units):
    print('{:4s} = {:5.2f} {}.'.format(v, x, u))

u_1  =  0.00 V.
u_2  =  1.00 V.
v_a  =  1.00 V.
v_R1 =  1.00 V.
v_R2 = -0.00 V.
v_cc =  0.00 V.
i_a  = -1.00 A.
i_R1 =  1.00 A.
i_R2 =  0.00 A.
i_cc =  1.00 A.


Vediamo ora se il circuito riportato sotto ammette soluzione:

<img src="figs/esistenza_unicita/ckt_2.png" width="400"/>

Le matrici $A$ e $M$ rimangono invariate, così come il vettore $\boldsymbol z$, mentre la matrice $N$ diventa:
    
$$
\begin{equation}
N = \left[
\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & -R_1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{array}
\right]
\end{equation}
$$

In [6]:
N_2 = np.array([[1,0,0,0],[0,-R1,0,0],[0,0,0,0],[0,0,0,0]])

Costruisco una nuova coppia $T$, $\boldsymbol y$ e verifico poi se $\mathrm{det}(T(t)) \neq 0$:

In [7]:
T_2,y_2 = make_T_y(A, M, N_2, z)
d_2 = np.linalg.det(T_2)
print('det(T) = {}'.format(d_2))

det(T) = 0.0


La matrice non è invertibile e di conseguenza non posso calcolare il vettore $\boldsymbol y$ delle soluzioni. Verifico con il teorema di Rouché-Capelli che effettivamente il risultato sia corretto:

In [8]:
def rouche_capelli(A, b):
    n_unknowns = A.shape[1]
    rnk_A = np.linalg.matrix_rank(A)
    rnk_Ab = np.linalg.matrix_rank(np.append(A, np.tile(b,(1,1)).T, axis=1))
    if rnk_A < rnk_Ab:
        print('rank(A) (= {}) < rank(A|b) (= {}): il sistema non ammette soluzioni.'.format(rnk_A, rnk_Ab))
    else:
        if rnk_A == n_unknowns:
            print('rank(A) = rank(A|b) = n = {}: il sistema ammette una sola soluzione.'.format(rnk_A))
        elif rnk_A < n_unknowns:
            print('{} = rank(A) = rank(A|b) < n = {}: il sistema ammette inf^({}) soluzioni.'\
                  .format(rnk_A, n_unknowns, n_unknowns-rnk_A))

In [9]:
rouche_capelli(T_2, y_2)

9 = rank(A) = rank(A|b) < n = 10: il sistema ammette inf^(1) soluzioni.


In [10]:
rouche_capelli(T, y)

rank(A) = rank(A|b) = n = 10: il sistema ammette una sola soluzione.
